In [11]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
%matplotlib inline

from ray import tune
import ray

In [12]:
ray.shutdown()
ray.init()
filename = "./data/wine.csv"
df = pd.read_csv(filename)
df = df.drop('index', axis=1)
print(df.shape)
print(df["quality"].value_counts())

2021-03-04 15:41:00,619	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(6497, 12)
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64


In [13]:
from sklearn.utils import resample

#Resample:
df_majority = df[df['quality'] == 6]
for i in range(3,10):
    majority_len = df[df['quality'] == 6].shape[0]
    if i != 6:
        minority_len = df[df['quality'] == i].shape[0]

        df_minority = df[df['quality'] == i]

        df_minority_upsampled = resample(df_minority, 
                                        replace=True,               
                                        n_samples=majority_len,
                                        random_state=123)

        df_majority = df_majority.append(df_minority_upsampled)
df = df_majority
print(df.shape)
print(df["quality"].value_counts())
df = df.sample(frac=1).reset_index(drop=True)

(19852, 12)
3    2836
4    2836
5    2836
6    2836
7    2836
8    2836
9    2836
Name: quality, dtype: int64


In [14]:
train_test_split_fraction = 0.80
split_index = int(df.shape[0] * train_test_split_fraction)
df_train = df[:split_index]
df_test = df[split_index:]

target = pd.get_dummies(df['quality']).values

X_train = df_train.drop('quality', axis = 1).values
X_test = df_test.drop('quality', axis = 1).values

y_train = target[:split_index]
y_test = target[split_index:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15881, 11)
(3971, 11)
(15881, 7)
(3971, 7)


In [15]:
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

In [16]:
#nb_hidden_neurons = 200
nb_classes = len(pd.unique(df['quality']))
nb_classes

7

In [17]:
import torch.nn as nn

class Network(nn.Module):

    def __init__(self, nb_features, nb_hidden_neurons):
        """Here we define the layers
        """

        super().__init__()
        
        self.layer_1 =  nn.Linear(nb_features, nb_hidden_neurons)
        self.layer_2 =  nn.Linear(nb_hidden_neurons, nb_hidden_neurons)
        self.layer_out =  nn.Linear(nb_hidden_neurons, nb_classes)
        
        

    def forward(self,x, nb_hidden_layers, activation_function):
        """Here we combine the layers
        """
        
        activation_function = activation_function 
        last_layer_activation = nn.Softmax()
        
        output_first_layer = activation_function(self.layer_1(x))
        outut_hidden_layer = activation_function(self.layer_2(output_first_layer))
        for i in range(1, nb_hidden_layers):
            outut_hidden_layer = activation_function(self.layer_2(outut_hidden_layer))
        prediction = last_layer_activation(self.layer_out(outut_hidden_layer))
        return prediction

In [18]:
def test(X, y, config):
    #my_nn = Network(nb_features=X_train.shape[1], nb_hidden_neurons=config['hidden_neuron'])
    correct = 0
    total = 0
    batch_size = config['batch_size']
    with torch.no_grad():
        for idx in range(0, X.shape[0], batch_size):
            
            total += 1
            
            output = my_nn(X.float(), nb_hidden_layers= config['nb_layers'],activation_function= config['activation'])
            
            predicted = torch.max(output, 1)
            
            target_pred = torch.max(y, 1)
            
            if predicted[1][idx] == target_pred[1][idx]:
                correct += 1
            
    accuracy = correct/total       
    return accuracy

In [19]:
def training(config):
    
    nb_steps_loss_sum = 10
    my_nn = Network(nb_features=X_train.shape[1], nb_hidden_neurons=config['hidden_neuron'])
    loss_list = []
    running_loss = 0
    batch_nb = 0
    criterion = nn.MSELoss()
    batch_size =  config['batch_size']
    learning_rate = config['lr'] 
    optimizer = torch.optim.AdamW(my_nn.parameters(), lr=learning_rate)

    for epoch in range(0,config['epoch']):
        for idx in range(0, X_train.shape[0], batch_size):
            
            input_batch = X_train[idx:idx + batch_size]
            target = y_train[idx:idx + batch_size]

            optimizer.zero_grad()

            y_pred = my_nn(input_batch.float(),nb_hidden_layers= config['nb_layers'],activation_function= config['activation'])

            loss = criterion(y_pred, target.float())

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            save_loss_condition = batch_nb % nb_steps_loss_sum == (nb_steps_loss_sum - 1)
            
            if save_loss_condition:    
                loss_list.append(running_loss)
                running_loss = 0.0

            batch_nb+= 1
    graph_loss = []
    graph_loss.append(loss_list)
    tune.report(mean_loss=abs(loss_list[len(loss_list)-1]))

In [20]:
search_space = {
    "lr": tune.grid_search([0.001, 0.01]),
    "hidden_neuron": tune.grid_search([50, 100, 200]),
    "epoch": tune.grid_search([20, 30, 60]),
    "batch_size": tune.grid_search([5, 30]),
    "nb_layers": tune.grid_search([5, 6, 7]),
    "activation": tune.grid_search([nn.SiLU()])
}
analysis = tune.run(training, config=search_space)

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_a4a08_00000,RUNNING,,SiLU(),5,20,50,0.001,5


Result for training_a4a08_00001:
  date: 2021-03-04_15-41-37
  done: false
  experiment_id: 0e83cb77ce0b4d0780cbf93a6ce63c58
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 0.6286819986999035
  neg_mean_loss: -0.6286819986999035
  node_ip: 192.168.1.42
  pid: 16591
  time_since_restore: 34.056334257125854
  time_this_iter_s: 34.056334257125854
  time_total_s: 34.056334257125854
  timestamp: 1614868897
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a4a08_00001
  


Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers,loss,iter,total time (s),neg_mean_loss
training_a4a08_00000,RUNNING,,SiLU(),5,20,50,0.001,5,,,,
training_a4a08_00001,RUNNING,192.168.1.42:16591,SiLU(),30,20,50,0.001,5,0.628682,1,34.0563,-0.628682
training_a4a08_00002,RUNNING,,SiLU(),5,30,50,0.001,5,,,,
training_a4a08_00003,RUNNING,,SiLU(),30,30,50,0.001,5,,,,
training_a4a08_00004,RUNNING,,SiLU(),5,60,50,0.001,5,,,,
training_a4a08_00005,RUNNING,,SiLU(),30,60,50,0.001,5,,,,
training_a4a08_00006,RUNNING,,SiLU(),5,20,100,0.001,5,,,,
training_a4a08_00007,RUNNING,,SiLU(),30,20,100,0.001,5,,,,
training_a4a08_00008,PENDING,,SiLU(),5,30,100,0.001,5,,,,


2021-03-04 15:41:38,000	INFO logger.py:690 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': SiLU()}
Result for training_a4a08_00001:
  date: 2021-03-04_15-41-37
  done: true
  experiment_id: 0e83cb77ce0b4d0780cbf93a6ce63c58
  experiment_tag: 1_activation=SiLU(),batch_size=30,epoch=20,hidden_neuron=50,lr=0.001,nb_layers=5
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 0.6286819986999035
  neg_mean_loss: -0.6286819986999035
  node_ip: 192.168.1.42
  pid: 16591
  time_since_restore: 34.056334257125854
  time_this_iter_s: 34.056334257125854
  time_total_s: 34.056334257125854
  timestamp: 1614868897
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a4a08_00001
  
Result for training_a4a08_00007:
  date: 2021-03-04_15-41-42
  done: false
  experiment_id: d68288402aec414a932756f8beb3ce84
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 0.5669487435370684
  neg_mean_loss: -0.5669487435370684
  node_ip: 192

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers,loss,iter,total time (s),neg_mean_loss
training_a4a08_00000,RUNNING,,SiLU(),5,20,50,0.001,5,,,,
training_a4a08_00002,RUNNING,,SiLU(),5,30,50,0.001,5,,,,
training_a4a08_00003,RUNNING,,SiLU(),30,30,50,0.001,5,,,,
training_a4a08_00004,RUNNING,,SiLU(),5,60,50,0.001,5,,,,
training_a4a08_00005,RUNNING,,SiLU(),30,60,50,0.001,5,,,,
training_a4a08_00006,RUNNING,,SiLU(),5,20,100,0.001,5,,,,
training_a4a08_00007,RUNNING,192.168.1.42:16587,SiLU(),30,20,100,0.001,5,0.566949,1,38.8432,-0.566949
training_a4a08_00008,RUNNING,,SiLU(),5,30,100,0.001,5,,,,
training_a4a08_00009,PENDING,,SiLU(),30,30,100,0.001,5,,,,
training_a4a08_00001,TERMINATED,,SiLU(),30,20,50,0.001,5,0.628682,1,34.0563,-0.628682


...

2021-03-04 16:50:30,511	INFO logger.py:690 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': SiLU()}
Result for training_a4a08_00100:
  date: 2021-03-04_16-50-30
  done: true
  experiment_id: db07236d656c44cd89e60f50aa89cfc6
  experiment_tag: 100_activation=SiLU(),batch_size=5,epoch=60,hidden_neuron=100,lr=0.01,nb_layers=7
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 2.400000050663948
  neg_mean_loss: -2.400000050663948
  node_ip: 192.168.1.42
  pid: 17416
  time_since_restore: 765.3189029693604
  time_this_iter_s: 765.3189029693604
  time_total_s: 765.3189029693604
  timestamp: 1614873030
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a4a08_00100
  
Result for training_a4a08_00106:
  date: 2021-03-04_16-54-31
  done: false
  experiment_id: c3fc68f944bd4c1e82cac02cb16b9798
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 2.400000050663948
  neg_mean_loss: -2.400000050663948
  node_ip: 192.168.1

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers,loss,iter,total time (s),neg_mean_loss
training_a4a08_00106,RUNNING,192.168.1.42:17513,SiLU(),5,60,200,0.01,7,2.4,1,721.862,-2.4
training_a4a08_00000,TERMINATED,,SiLU(),5,20,50,0.001,5,0.518528,1,192.791,-0.518528
training_a4a08_00001,TERMINATED,,SiLU(),30,20,50,0.001,5,0.628682,1,34.0563,-0.628682
training_a4a08_00002,TERMINATED,,SiLU(),5,30,50,0.001,5,0.481678,1,296.068,-0.481678
training_a4a08_00003,TERMINATED,,SiLU(),30,30,50,0.001,5,0.566713,1,51.4407,-0.566713
training_a4a08_00004,TERMINATED,,SiLU(),5,60,50,0.001,5,0.527847,1,604.466,-0.527847
training_a4a08_00005,TERMINATED,,SiLU(),30,60,50,0.001,5,0.412463,1,98.3762,-0.412463
training_a4a08_00006,TERMINATED,,SiLU(),5,20,100,0.001,5,0.676416,1,206.065,-0.676416
training_a4a08_00007,TERMINATED,,SiLU(),30,20,100,0.001,5,0.566949,1,38.8432,-0.566949
training_a4a08_00008,TERMINATED,,SiLU(),5,30,100,0.001,5,0.549177,1,318.707,-0.549177


2021-03-04 16:54:31,539	INFO logger.py:690 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': SiLU()}
Result for training_a4a08_00106:
  date: 2021-03-04_16-54-31
  done: true
  experiment_id: c3fc68f944bd4c1e82cac02cb16b9798
  experiment_tag: 106_activation=SiLU(),batch_size=5,epoch=60,hidden_neuron=200,lr=0.01,nb_layers=7
  hostname: iBoo.lan
  iterations_since_restore: 1
  mean_loss: 2.400000050663948
  neg_mean_loss: -2.400000050663948
  node_ip: 192.168.1.42
  pid: 17513
  time_since_restore: 721.8616409301758
  time_this_iter_s: 721.8616409301758
  time_total_s: 721.8616409301758
  timestamp: 1614873271
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a4a08_00106
  


Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers,loss,iter,total time (s),neg_mean_loss
training_a4a08_00000,TERMINATED,,SiLU(),5,20,50,0.001,5,0.518528,1,192.791,-0.518528
training_a4a08_00001,TERMINATED,,SiLU(),30,20,50,0.001,5,0.628682,1,34.0563,-0.628682
training_a4a08_00002,TERMINATED,,SiLU(),5,30,50,0.001,5,0.481678,1,296.068,-0.481678
training_a4a08_00003,TERMINATED,,SiLU(),30,30,50,0.001,5,0.566713,1,51.4407,-0.566713
training_a4a08_00004,TERMINATED,,SiLU(),5,60,50,0.001,5,0.527847,1,604.466,-0.527847
training_a4a08_00005,TERMINATED,,SiLU(),30,60,50,0.001,5,0.412463,1,98.3762,-0.412463
training_a4a08_00006,TERMINATED,,SiLU(),5,20,100,0.001,5,0.676416,1,206.065,-0.676416
training_a4a08_00007,TERMINATED,,SiLU(),30,20,100,0.001,5,0.566949,1,38.8432,-0.566949
training_a4a08_00008,TERMINATED,,SiLU(),5,30,100,0.001,5,0.549177,1,318.707,-0.549177
training_a4a08_00009,TERMINATED,,SiLU(),30,30,100,0.001,5,0.474097,1,53.6717,-0.474097


2021-03-04 16:54:31,586	INFO tune.py:450 -- Total run time: 4409.05 seconds (4408.97 seconds for the tuning loop).


In [21]:
best = analysis.get_best_config(metric="mean_loss", mode="min")
print(best)

{'lr': 0.001, 'hidden_neuron': 200, 'epoch': 60, 'batch_size': 30, 'nb_layers': 7, 'activation': SiLU()}


In [23]:
acc_train = test(X_train, y_train, best)
acc_test = test(X_test, y_test, best)
print(acc_test)
print(acc_train)

0.07518796992481203
0.1339622641509434
